In [1]:
import numpy as np
import config

import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

import absl.logging
absl.logging.set_verbosity(absl.logging.ERROR)

In [2]:
from tensorflow import keras

def create_model():
    print('Creating new model...', end='\r')
    
    Resnet_base = keras.applications.resnet50.ResNet50(include_top=False, input_shape=config.input_shape)
    for layer in Resnet_base.layers:
        layer.trainable = False
    
    input_layer = keras.Input(shape=config.input_shape)
    Resnet = keras.applications.resnet.preprocess_input(input_layer)
    Resnet = Resnet_base(Resnet)
    Resnet = keras.layers.GlobalAveragePooling2D()(Resnet)
    
    net = keras.layers.Dense(units=4096)(Resnet)
    net = keras.layers.BatchNormalization()(net)
    net = keras.layers.Activation(keras.activations.relu)(net)
    net = keras.layers.Dropout(0.4)(net)
    
    net = keras.layers.Dense(units=2048)(net)
    net = keras.layers.BatchNormalization()(net)
    net = keras.layers.Activation(keras.activations.relu)(net)
    net = keras.layers.Dropout(0.4)(net)
    
    output_layer = keras.layers.Dense(units=config.output_shape, activation='softmax')(net)
    
    model=keras.Model(inputs=[input_layer],outputs=[output_layer])
    model.compile(loss='categorical_crossentropy', optimizer=keras.optimizers.Adam(learning_rate=3e-4), metrics=[keras.metrics.CategoricalAccuracy()])
    
    print('Successfully created new model.')
    return model

def load_model(dir):
    print('Loading model from %s'%dir, end='\r')
    model = keras.models.load_model(dir)
    print('Successfully loaded model from %s'%dir)
    return model

def save_model(dir, model):
    print('Saving model to %s'%dir, end='\r')
    model.save(dir)
    print('Successfully saved model to %s'%dir)

def save_plot(dir, model):
    print('Saving model plot to %s'%dir, end='\r')
    keras.utils.plot_model(model,dir, show_shapes=True, expand_nested=True)
    print('Successfully saved model plot to %s'%dir)

def save_summary(dir, model):
    print('Saving model summary to %s'%dir, end='\r')
    with open(dir, 'w') as f:
        model.summary(print_fn=lambda x:f.write(x + '\n'), expand_nested=True)
    print('Successfully saved model summary to %s'%dir)

In [ ]:
augmentation = keras.Sequential(
    [
        keras.layers.RandomFlip(mode='horizontal'),
        keras.layers.RandomZoom(height_factor=(-0.1,0.1)),
        keras.layers.RandomTranslation(height_factor=(-0.1,0.1), width_factor=(-0.1,0.1)),
        keras.layers.RandomRotation(factor = (0.1))
    ]
)

In [3]:
def create_index(source):
    return (source, os.listdir(source))

def generator(source, index, num_batch, data_augmentation=True):
    while(True):
        np.random.shuffle(index)
        for i in range(num_batch):
            X, y = np.load( os.path.join(source, index[i]), allow_pickle=True )
            X = np.asarray( list(X), 'float32' )
            if(data_augmentation):
                X = augmentation(X)
            y = np.asarray( list(y), 'float32' )
            yield(X, y)    

def create_generator(index, data_augmentation=True):
    source = index[0]
    index = index[1]
    num_batch = len(index)
    gen = generator(source, index, num_batch, data_augmentation)
    return gen, num_batch

In [4]:
model_name = 'v2-resnet-50-personal'

In [ ]:
# model = create_model()
model = load_model( os.path.join(config.dir_model_saved, '%s/'%model_name) )

save_plot( os.path.join(config.dir_model_plot, '%s-plot.png'%model_name), model )
save_summary( os.path.join(config.dir_model_summary, '%s-summary.txt'%model_name), model )

In [ ]:
train_index = create_index(config.train_personal_batch)
val_index = create_index(config.val_personal_batch)

train_gen, train_num_batch = create_generator(train_index)
val_gen, val_num_batch = create_generator(val_index, data_augmentation=False)

In [ ]:
from keras.callbacks import ModelCheckpoint
checkpoint = ModelCheckpoint(filepath=os.path.join(config.dir_model_saved, '%s/'%model_name), monitor='val_loss', verbose=1, save_best_only=True, mode='min')
callbacks = [checkpoint]

history = model.fit(
    x = train_gen,
    steps_per_epoch = train_num_batch,
    validation_data = val_gen,
    validation_steps = val_num_batch,
    epochs = 30,
    verbose = 1,
    callbacks = callbacks
)

In [5]:
test_index = create_index(config.test_batch)
test_gen, test_num_batch = create_generator(test_index, data_augmentation=False)

In [6]:
test_model = load_model( os.path.join(config.dir_model_saved, '%s/'%model_name) )
result = test_model.evaluate(
    x = test_gen,
    steps = test_num_batch,
    verbose = 1
)

Successfully loaded model from ../Resources/model_saved/v2-resnet-50-personal/
225/225 [==============================] - 86s 365ms/step - loss: 3.3441 - categorical_accuracy: 0.2935
